In [2]:
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import cv2
import random
import os
import csv

## First step is to load in the data

path_box = "../labeled-data/Annotations/Annotations/daySequence1/frameAnnotationsBOX.csv"
path_bulb = "../labeled-data/Annotations/Annotations/daySequence1/frameAnnotationsBULB.csv"
frames_path = "../labeled-data/daySequence1/daySequence1/frames/"


file_paths=[]
frame_ids=[]
for filename in os.listdir(frames_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        file_paths.append(os.path.join(frames_path, filename))
        frame_ids.append(filename)
        
    else:
        continue
        
frame_annotations = {}


with open(path_box) as fp:  
    line = fp.readline()
    line = fp.readline() # Skip header line with descriptions
    while line:
        annotation = (line.strip()).split(";")
        frame_id = annotation[0].split("/")[1]
        if frame_id in frame_annotations:
            frame_annotations[frame_id].append(annotation)
        else:
            frame_annotations[frame_id] = [annotation]
        line = fp.readline()
        

with open('../labeled-data/labels.csv', 'w', newline='') as csvfile: 
    writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(len(file_paths)):
        img = cv2.imread(file_paths[i])
        frame_id = frame_ids[i]
        if frame_id in frame_annotations:
            annotations = frame_annotations[frame_id]
            for j,anno in enumerate(annotations):
                anno_class = anno[1]
                anno_left = int(anno[2])
                anno_top = int(anno[3])
                anno_right = int(anno[4])
                anno_bot = int(anno[5])
                #print("\tClass: '{}' at [{},{},{},{}]".format(anno_class, anno_left, anno_top, anno_right, anno_bot))
                crop_img = img[anno_top:anno_bot, anno_left:anno_right]
                filename = 'img_' + str(i) + '_'+ str(j) +'.jpg'
                writer.writerow([filename,anno_class])
                cv2.imwrite(os.path.join('../labeled-data/segmented-images',filename), crop_img)

